# Tensorflow

Tensorflow is a deep learning framework, especially to use the GPU power and  its capacities of distribute processing. In this you can find tensors (they are a specially array) and constants (they are simple tensors).

In order to use the capacities of distributed computing, first, it is important to create a session.


In [3]:
import tensorflow as tf

c = tf.constant(5.0)

print (c)

Tensor("Const_2:0", shape=(), dtype=float32)


In [4]:
sess =  tf.Session()
sess.run(c)

5.0

## Placeholder

A placeholder is the mechanism to call/use functions in a session

In [8]:
x = tf.placeholder(tf.float32,())
y = x * 2.0

print(y)

#dictionary of inputs
values = {x: 9.0}

print(float(sess.run(y,values)))

Tensor("mul_3:0", shape=(), dtype=float32)
18.0
